In [2]:
#I'm not sure why people makes their Fourier Positional Encoding so complicate and I'm not sure they are right. So here is my own version

%%capture
!pip install einops

In [3]:
from torchvision.io import read_image
from einops import rearrange
import torch
import math

In [56]:
im = read_image('/content/224_224_image.jpg')
print('image size:',im.size())

image size: torch.Size([3, 224, 224])


In [57]:
# Flatten the image from a 3 dimension to 2 dimension
flatten_im = rearrange(im,'c h w -> c (h w)')
print('flatten image size:',flatten_im.size())

flatten image size: torch.Size([3, 50176])


In [58]:
# Copy each image channel
x1 = flatten_im[0]
x2 = flatten_im[1]
x3 = flatten_im[2]

In [59]:
# create a function to normalise the image array between -1 and 1
def normalisation(x):
  x =  x/x.sum(0).expand_as(x) 
  x[torch.isnan(x)]=0   #if an entire column is zero, division by 0 will cause NaNs
  x = 2*x - 1
  return x

In [60]:
#Normalise each channel
x1 = normalisation(x1)
x2 = normalisation(x2)
x3 = normalisation(x3)

In [61]:
#Put all channel back together
normalised_image = torch.cat((x1,x2,x3),dim=0).view(3,-1)

In [73]:
#create a log splace tensor between 1 and pi/2
fk_sin = torch.logspace(start=1, end=0.5*math.pi, steps=7, base=2)
fk_cos = torch.logspace(start=1, end=0.5*math.pi, steps=7, base=2)
pe = torch.zeros(3,224*224,7*2+1)

In [79]:
# Create a fourier positional encoding
for j in range(2):
  for i in range(len(normalised_image[j])):
    x = normalised_image[j][i]
    sin = torch.sin(math.pi*x*fk_sin)
    cos = torch.cos(math.pi*x*fk_cos)
    fk = torch.cat((sin,cos))
    fk_ = fk.tolist()
    fk_.append(x.item())
    fk = torch.tensor(fk_)
    pe[j][i]=fk

In [80]:
pe.size()

torch.Size([3, 50176, 15])

In [86]:
#Explaination:
#dth dimension: 0
d0 = pe[0]
# Along this dimension, there are 50176 inputs, which is equivalent to the flatten image array of 224 x 224
print('Number of inputs in each dimension: ',d0.size(0))
# Within each input, there are 15 values, these 15 values corresponds to 2K + 1. Where K is the number of bands. In this example, a value of 7 was used. The reason you times K by 2 is because
#you need to calculate the cos and sin value for each band frequency. You add 1 to the end because you need to concat the normalised input with the 14 fourier values (7*2+1 = 15)
print('Number of positional encoding values for each input:',d0.size(1))

Number of inputs in each dimension:  50176
Number of positional encoding values for each input: 15
